# Explore HSR data
Stefan/Yuzhao Heng
Since Tues. Oct. 12th, 2021

Check the types, format and content for HSR robot's laser scan output, see `scripts/util` for the type.


## Setup

In [82]:
import os
import json
import numpy as np
from collections.abc import Iterable
from math import isclose
from icecream import ic

from util import *

os.chdir('../data')

fnms = !ls

def get_scans(fnm):
    f = open(fnm, 'r')
    scans = json.load(f)
    f.close()
    return scans

## A rather static recording
Note that each object in the JSON file is a list of LaserScan objects.



In [83]:
scans = get_scans(fnms[0])


## Check attributes in single scan



In [97]:
s = scans[0]

def pp(scan, n=10, w=2):
    ks, vs = [], []
    for k, v in scan.items():
        if type(v) is dict:
            for k_, v_ in v.items():
                ks.append(f'{k}.{k_}')
                vs.append(v_)
        else:
            ks.append(k)
            vs.append(v[:n] if isinstance(v, Iterable) else v)

    for k, v in zip(ks, vs):
        print(f'{k: <20} {v}')

pp(s)

n_beams = ((s['angle_max'] - s['angle_min']) / s['angle_increment']) + 1
assert isclose(n_beams, int(n_beams), abs_tol=1e-4)
n_beams = int(n_beams)
ic(n_beams)

intens = np.array(s['intensities'])
ranges = np.array(s['ranges'])
assert n_beams == intens.size and n_beams == ranges.size

ic(intens[:10], ranges[:10])

ic| n_beams: 963
ic| intens[:10]: array([ 827., 1088., 1402., 1540., 1679., 1681., 1724., 1749., 1825.,
                        1863.])
    ranges[:10]: array([1.71099997, 1.75100005, 1.778     , 1.78499997, 1.79700005,
                        1.81500006, 1.824     , 1.829     , 1.83200002, 1.83200002])


header.seq           134710
header.stamp         {'secs': 1633995068, 'nsecs': 214782575}
header.frame_id      base_range_sensor_link
angle_min            -2.0987584590911865
angle_max            2.0987584590911865
angle_increment      0.004363323096185923
time_increment       1.736111516947858e-05
scan_time            0.02500000037252903
range_min            0.019999999552965164
ranges               [1.7109999656677246, 1.7510000467300415, 1.777999997138977, 1.784999966621399, 1.7970000505447388, 1.815000057220459, 1.8240000009536743, 1.8289999961853027, 1.8320000171661377, 1.8320000171661377]
intensities          [827.0, 1088.0, 1402.0, 1540.0, 1679.0, 1681.0, 1724.0, 1749.0, 1825.0, 1863.0]


(array([ 827., 1088., 1402., 1540., 1679., 1681., 1724., 1749., 1825.,
        1863.]),
 array([1.71099997, 1.75100005, 1.778     , 1.78499997, 1.79700005,
        1.81500006, 1.824     , 1.829     , 1.83200002, 1.83200002]))

## Across consecutive scans



In [85]:
ic(len(scans))

def map_scans(key):
    key = key.split('.')
    return np.vectorize(lambda x: get(x, key))(scans)


frame_ids = map_scans('header.frame_id')
seqs = map_scans('header.seq')
secs = map_scans('header.stamp.secs')
nsecs = map_scans('header.stamp.nsecs')
range_mins = map_scans('range_min')
scan_times = map_scans('scan_time')
time_increments = map_scans('time_increment')
ic(frame_ids, seqs, secs, nsecs, range_mins, scan_times, time_increments)


ic| len(scans): 106
ic| frame_ids: array(['base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                      'base_range_sensor_link', 'base_range_sensor_link',
                  

(array(['base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_range_sensor_link',
        'base_range_sensor_link', 'base_